In [1]:
import ROOT
import os, sys

Welcome to JupyROOT 6.26/10


In [2]:
campaign = 'UL2018'
filelist = os.listdir(f'inputs/{campaign}')
filelist = [item for item in filelist if item.endswith("root")] #Keeping only the file that end with 'root'
outdir = f'plots/{campaign}/'
os.system(f'mkdir -p {outdir}')
print('files read.')

files read.


### Producing the efficiency maps from each sample and storing the numbers in a dictionary

In [3]:
def produce_eff_dict(type):
    eff_dict = {}
    
    for file in filelist:   
        sample = file.split('_')[1]
        subsample = file.split('_')[2].split('.')[0]
        #Correcting for QCD and VLLs
        if 'QCD' in file or 'VLL' in file:
            sample = file.split('_')[1]+'_'+file.split('_')[2]
            subsample = file.split('_')[3].split('.')[0]
    
        #print(f'Producing the scale factors for {sample} {subsample}')
        eff_dict[sample+"_"+subsample] = {}
    
        #Reading the file as TFile and extracting the 2D histograms:
        filepath = f'inputs/{campaign}/{file}'
        tfile = ROOT.TFile.Open(filepath, 'READ')
    
        if type == 'bJetTagEff':
            num = 'bJet_MedWP_PtEta'
            den = 'bJet_PtEta'
        elif type == 'cJetMisEff':
            num = 'cJet_Mis_PtEta' 
            den = 'cJet_PtEta'
        elif type == 'lightJetMisEff':
            num = 'LightJet_Mis_PtEta' 
            den = 'LightJet_PtEta'
    
        hst_num = tfile.Get(num)
        hst_den = tfile.Get(den)
    
        #Producing the efficiency histogram:
        hst_eff=hst_num.Clone()
        hst_eff.Divide(hst_den)

        bin = 0
        for y in range(1, 4):
            eff_dict[sample+"_"+subsample][f'etabin_{y}'] = {}
            for x in range(2, 10):
                bin=bin+1
                eff_dict[sample+"_"+subsample][f'etabin_{y}'][f'ptbin_{x}'] = {}
                eff = hst_eff.GetBinContent(x, y)
                efferr = hst_eff.GetBinError(x, y)
                eff_dict[sample+"_"+subsample][f'etabin_{y}'][f'ptbin_{x}']['eff'] = eff
                eff_dict[sample+"_"+subsample][f'etabin_{y}'][f'ptbin_{x}']['efferr'] = efferr
                #print(x, y, eff)
            
        #Making a plot for the efficieency bins
        canvas=ROOT.TCanvas('effmap_'+type+'_'+sample+'_'+subsample, "", 800, 600)
        ROOT.gStyle.SetOptStat(0)
    
        #Decorating the hist
        hst_eff.SetTitle(type+' for '+sample+' '+subsample)
        hst_eff.SetTitleSize(1)
        hst_eff.SetTitleOffset(0.8)
        hst_eff.Draw('colz text same')
        hst_eff.GetZaxis().SetRangeUser(0.0,1.0)
        hst_eff.GetYaxis().SetTitle("|#eta|")
        hst_eff.GetYaxis().SetTitleSize(0.05)
        hst_eff.GetYaxis().SetTitleOffset(0.8)
        hst_eff.GetYaxis().SetLabelSize(0.03)
        hst_eff.GetXaxis().SetTitle("p_{T}   ") 
        hst_eff.GetXaxis().SetTitleSize(0.05)
        hst_eff.GetXaxis().SetTitleOffset(0.8)
        hst_eff.GetXaxis().SetLabelSize(0.03)
        #Decorating the canvas
        canvas.SetLogx(1)
        ROOT.gStyle.SetPaintTextFormat(".2f");
        canvas.Draw()
        os.system(f'mkdir -p {outdir}/{type}')
        #print(f'{outdir}{type}/{type}_{sample}_{subsample}.png')
        canvas.SaveAs(f'{outdir}{type}/{den}_{sample}_{subsample}.png')
        #print(f'nbins = {bin}')
        #break

    return eff_dict

In [4]:
eff_dict_bTag = produce_eff_dict('bJetTagEff')
eff_dict_cMis = produce_eff_dict('cJetMisEff')
eff_dict_lMis = produce_eff_dict('lightJetMisEff')

Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_DYJetsToLL_M10to50.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_DYJetsToLL_M50.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_HTbinnedWJets_100to200.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_HTbinnedWJets_1200to2500.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_HTbinnedWJets_200to400.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_HTbinnedWJets_2500toInf.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_HTbinnedWJets_400to600.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_HTbinnedWJets_600to800.png has been created
Info in <TCanvas::Print>: png file plots/UL2018/bJetTagEff/bJet_PtEta_HTbinnedWJets_70to100.png h

### Reading from the dictinory and prining/saving the numbers (simple text files)

In [5]:
def generate_texts(type):
    eff_dict = None
    if type == 'bJetTagEff' :       eff_dict = eff_dict_bTag
    elif type == 'cJetMisEff' :     eff_dict = eff_dict_cMis
    elif type == 'lightJetMisEff' : eff_dict = eff_dict_lMis
    
    lines_of_texts = [f'{type} summary for {campaign}', '']
    for sample, bindict in eff_dict.items():
        #print(f'//For the sample {sample}:')
        lines_of_texts += [f"{sample}", ' ']

        list_etabins = []
        for etabins, valuedict in sorted(bindict.items()):
            #print(etabins)
            list_ptbins = []
            for ptbins, value in sorted(valuedict.items()):
                #print(ptbins, value['eff'])
                list_ptbins.append(value['eff'])
            list_etabins.append(list_ptbins)

    list_str = str(list_etabins)
    list_str = list_str.replace('[', '{').replace(']', '}')
    lines_of_texts+=([list_str, ''])
    #print(list_str)
    print('Done')
    return list_str

In [6]:
txt_bTag = generate_texts('bJetTagEff')
txt_cMis = generate_texts('cJetMisEff')
txt_lMis = generate_texts('lightJetMisEff')

Done
Done
Done


In [7]:
def save_text_file(type):
    with open(f"text_files/{campaign}_{type}.txt", "w") as file:
    # Iterate over each line in lines_of_texts
        lines_of_texts = None
        if type == 'bJetTagEff' :       lines_of_texts = txt_bTag
        elif type == 'cJetMisEff' :     lines_of_texts = txt_cMis
        elif type == 'lightJetMisEff' : lines_of_texts = txt_lMis
        for line in lines_of_texts:
            file.write(line)  
            file.write("\n")
    print("Done!")

In [8]:
save_text_file('bJetTagEff')
save_text_file('cJetMisEff')
save_text_file('lightJetMisEff')

Done!
Done!
Done!


### Generate code snippet and save it as a header file: (be careful!)

In [9]:
raw_strings = []

beginning1 = r'''#include <vector>
#include "TMath.h"
#include "/home/work/phazarik1/work/Analysis-Run3/Setup/Corrections/bJetCorrections/scalefactor_bTagJets.h"
using namespace std;

// bTag SF for the event (looped over all jets)
// For "data" efficiency, enter the stored JetDeepMediumCSVSF value
// For "mc" efficiency, enter JetDeepMediumCSVSF as 1.0 !!

// ------------------------------------------------------------------------------------------
// ------------------------------------------------------------------------------------------

'''
function_name = f'float AnaScript::bTagEff_{campaign}(vector<Particle>Jet)'+'{'

beginning2 = r'''

  //initializing
  float probability_mc  = 1.0;
  float probability_data = 1.0;

  float LJetEff[3][8] = {{1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0},  //eta: 0.0-0.8
			 {1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0},  //eta: 0.8-1.6
			 {1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0}}; //eta: 1.6-2.4

  float CJetEff[3][8] = {{1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0},
			 {1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0},
			 {1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0}};

  float BJetEff[3][8] = {{1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0},
			 {1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0},
			 {1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0}};
'''
raw_strings.append(beginning1)
raw_strings.append(function_name)
raw_strings.append(beginning2)

for sample, bindict in eff_dict_bTag.items():
    #print(f'\n//For the sample {sample}:')
    eta_bins = eff_dict_bTag[sample].keys()

    list_bTag_etabins=[]
    list_cMis_etabins=[]
    list_lMis_etabins=[]
    for eta in eta_bins:
        list_btag_ptbins=[]
        list_cMis_ptbins=[]
        list_lMis_ptbins=[]
        pt_bins = eff_dict_bTag[sample][eta].keys()
        for pt in pt_bins:
            list_btag_ptbins.append(eff_dict_bTag[sample][eta][pt]['eff'])
            list_cMis_ptbins.append(eff_dict_cMis[sample][eta][pt]['eff'])
            list_lMis_ptbins.append(eff_dict_lMis[sample][eta][pt]['eff'])
        list_bTag_etabins.append(list_btag_ptbins)
        list_cMis_etabins.append(list_cMis_ptbins)
        list_lMis_etabins.append(list_cMis_ptbins)

    #Writing the code block with proper indentation:
    if_statement = f'\n  //For the sample {sample}:\n' + f'  if(_samplename == "{sample}")'+'{'
    list_bTag = "    float BJetEff_temp[3][8] = "+str(list_bTag_etabins).replace('[', '{').replace(']', '}')+";"
    list_cMis = "    float CJetEff_temp[3][8] = "+str(list_cMis_etabins).replace('[', '{').replace(']', '}')+";"
    list_lMis = "    float LJetEff_temp[3][8] = "+str(list_lMis_etabins).replace('[', '{').replace(']', '}')+";"
    for_loop = r'''
    for(int i=0; i<3 ; i++){
      for(int j=0; j<8; j++){
        LJetEff[i][j] = LJetEff_temp[i][j];
        CJetEff[i][j] = CJetEff_temp[i][j];
        BJetEff[i][j] = BJetEff_temp[i][j];
      }
    }
  }
    '''
    
    code_block = f'''{if_statement}
{list_bTag}
{list_cMis}
{list_lMis}
{for_loop}
    '''
    raw_strings.append(code_block)

#Code block is added to the raw string for each sample.
#Now, add the rest:
ending = r'''
  //---------------------------------------
  // Correction to each jet:
  //---------------------------------------
  
  for (int i = 0; i < (int)Jet.size(); i++){
    float jet_prob_mc = 1.0;
    float jet_prob_data = 1.0;
    float jet_eff = 1.0;
    // choose pt bin
    int ptbin=-1;
    if     ( Jet.at(i).v.Pt() < 50  )  ptbin=0;
    else if( Jet.at(i).v.Pt() < 70  )  ptbin=1;
    else if( Jet.at(i).v.Pt() < 100 )  ptbin=2;
    else if( Jet.at(i).v.Pt() < 140 )  ptbin=3;
    else if( Jet.at(i).v.Pt() < 200 )  ptbin=4;
    else if( Jet.at(i).v.Pt() < 300 )  ptbin=5;
    else if( Jet.at(i).v.Pt() < 600 )  ptbin=6;
    else                               ptbin=7;
    // choose eta bin
    int etabin=-1;
    if      ( fabs(Jet.at(i).v.Eta()) < 0.8 ) etabin=0;
    else if ( fabs(Jet.at(i).v.Eta()) < 1.6 ) etabin=1;
    else if ( fabs(Jet.at(i).v.Eta()) < 2.4 ) etabin=2;
    else                                 continue;

    //Setting the appropriate numbers for the jet:
    // choose flavor  
    if      ( Jet.at(i).hadronflavor == 5 ) jet_eff = BJetEff[etabin][ptbin];
    else if ( Jet.at(i).hadronflavor == 4 ) jet_eff = CJetEff[etabin][ptbin];
    else                                    jet_eff = LJetEff[etabin][ptbin];

    double SFfromPOG =  1.0;
    if( Jet.at(i).hadronflavor == 5)       SFfromPOG = getScaleFactors_bTagJets_MedWP_UL18(Jet.at(i).v.Eta(),Jet.at(i).v.Pt());
    else if(Jet.at(i).hadronflavor == 4 )  SFfromPOG = getScaleFactors_cTagJets_Mis_UL18(Jet.at(i).v.Eta(),Jet.at(i).v.Pt());
    else                                   SFfromPOG = getScaleFactors_LightTagJets_Mis_UL18(Jet.at(i).v.Eta(),Jet.at(i).v.Pt());
    //These functions are defined in the other file
    
    // check if jet is tagged or not
    if(Jet_btagDeepB[Jet.at(i).ind] > 0.4168){
      jet_prob_data = jet_eff*SFfromPOG;
      jet_prob_mc   = jet_eff;
    }
    else {    
      jet_prob_data = 1-jet_eff*SFfromPOG;
      jet_prob_mc   = 1-jet_eff;
    }
    
    probability_data *= jet_prob_data;
    probability_mc   *= jet_prob_mc;
  }
  
  float scaleFactor = 1.0;
  if (probability_mc > 0.0) scaleFactor = probability_data/probability_mc;
  
  return scaleFactor;
}
'''

raw_strings.append(ending)

# Writing these to a text file:
file_name = f"text_files/JetEff_DeepJet_MediumWP_{campaign}.h"
with open(file_name, "w") as file:
    for each_block in raw_strings:
        file.write(each_block)

# Confirm the writing process
print(f"Code blocks have been written to {file_name}")    

Code blocks have been written to text_files/JetEff_DeepJet_MediumWP_UL2018.h
